In [ ]:
include("../src/Kraft.jl")

In [ ]:
Kraft.check_programs()

In [ ]:
const path_dict = Kraft.make_path_dict(
    "/home/kwat/data",
    1,
)

In [ ]:
const germ_dna_1_fq_gz = "/home/kwat/sample/germ_dna.1.fastq.gz"

const germ_dna_2_fq_gz = "/home/kwat/sample/germ_dna.2.fastq.gz"

const soma_dna_1_fq_gz = "/home/kwat/sample/soma_dna.1.fastq.gz"

const soma_dna_2_fq_gz = "/home/kwat/sample/soma_dna.2.fastq.gz"

const exome = true

const output_dir = "/home/kwat/sample/process_soma_dna"

const n_job = 8

const gb_memory = 56

const job_gb_memory = 6

In [ ]:
for file_path::String in(
    germ_dna_1_fq_gz,
    germ_dna_2_fq_gz,
    soma_dna_1_fq_gz,
    soma_dna_2_fq_gz,
)
    
    if !isfile(file_path)
        
        error("$file_path doesn't exist.")
    
    end
    
end

In [ ]:
using Dates

In [ ]:
start_time = now()

In [ ]:
const trim_sequence_prefix = joinpath(
    output_dir,
    "trim_sequence",
    "germ",
)

Kraft.trim_sequence(
    germ_dna_1_fq_gz,
    germ_dna_2_fq_gz,
    trim_sequence_prefix,
    n_job,
)

const germ_trim_1_fq_gz = "$trim_sequence_prefix-trimmed-pair1.fastq.gz"

const germ_trim_2_fq_gz = replace(
    germ_trim_1_fq_gz,
    "pair1"=>"pair2",
)

In [ ]:
const trim_sequence_prefix = joinpath(
    output_dir,
    "trim_sequence",
    "soma",
)

Kraft.trim_sequence(
    soma_dna_1_fq_gz,
    soma_dna_2_fq_gz,
    trim_sequence_prefix,
    n_job,
)

const soma_trim_1_fq_gz = "$trim_sequence_prefix-trimmed-pair1.fastq.gz"

const soma_trim_2_fq_gz = replace(
    soma_trim_1_fq_gz,
    "pair1"=>"pair2",
)

In [ ]:
Kraft.check_sequence(
    (germ_trim_1_fq_gz, germ_trim_2_fq_gz, soma_trim_1_fq_gz, soma_trim_2_fq_gz),
    joinpath(output_dir, "check_sequence"),
    n_job,
)

In [ ]:
const germ_bam = joinpath(
    output_dir,
    "align_sequence",
    "germ_dna.bam",
)

Kraft.align_sequence(
    germ_trim_1_fq_gz,
    germ_trim_2_fq_gz,
    "Germ",
    path_dict["dna.fa.gz"],
    germ_bam,
    n_job,
    job_gb_memory,
)

In [ ]:
const soma_bam = joinpath(
    output_dir,
    "align_sequence",
    "soma_dna.bam",
)

Kraft.align_sequence(
    soma_trim_1_fq_gz,
    soma_trim_2_fq_gz,
    "Soma",
    path_dict["dna.fa.gz"],
    soma_bam,
    n_job,
    job_gb_memory,
)

In [ ]:
const find_mutation_dir = joinpath(
    output_dir,
    "find_mutation",
)

Kraft.find_mutation(
    germ_bam,
    soma_bam,
    path_dict["dna.fa.bgz"],
    path_dict["chromosome.bed.gz"],
    exome,
    path_dict["chrn_n.tsv"],
    find_mutation_dir,
    n_job,
    gb_memory,
)

In [ ]:
end_time = now()

In [ ]:
canonicalize(Dates.CompoundPeriod(end_time - start_time))